LSTM MODEL
Yufan qian

In [3]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import nltk
import math
from torch.utils.data import Dataset, DataLoader
from collections import Counter


In [4]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [24]:
glove_file = '../glove.6B/glove.6B.50d.txt'

embeddings_dict = {}

with open(glove_file, 'r', encoding='utf8') as f:
    for i, line in enumerate(f):
        line = line.strip().split(' ')
        word = line[0]
        embed = np.asarray(line[1:], "float")

        embeddings_dict[word] = embed


print('Loaded {} words from glove'.format(len(embeddings_dict)))

low = -1.0 / 3
high = 1.0 / 3
embedding_matrix = np.random.uniform(low=low, high=high, size=(len(embeddings_dict)+1, 50))

word2id = {}
for i, word in enumerate(embeddings_dict.keys(), 1):

    word2id[word] = i                                
    embedding_matrix[i] = embeddings_dict[word]      

word2id['<pad>'] = 0

Loaded 400000 words from glove


In [25]:
class dataset(Dataset):
    def __init__(self, csv_file, max=2000):
        self.data = pd.read_csv(csv_file)
        
        self.tokenize = get_tokenize
#         for text in self.data['text']:
#             min_fre = 1
#             self.vob = vob(counter)
    def __len__(self):
        return len(self.data)
    def __getvob__(self,index):
        label = self.data.iloc[index]['labels']
        content = content[:self.max]
        content_tokens = [self.vocab[token] for token in self.token(content)]
        if len(content_tokens) < self.max:
            content_tokens += [0]*(self.max - len(content_tokens))
        else:
            content_tokens = content_tokens[:self.max]
        
        content_tensor = torch.tensor(content_tokens)
        
        labels_tensor = torch.tensor(1 if labels == 'spam' else 0)
    
        return {'text': content_tensor, 'labels': labels_tensor}
    def __getitem__(self,index):
#         print(index)
#         print(self.data.head())
        return (self.data.iloc[index,2],self.data.iloc[index,3])

        

In [26]:
print(type(spam_train))
spam_train = dataset('./datasets/SpamHam/train.csv')
print(spam_train[20])

<class '__main__.dataset'>
('more auto terminate deals original message from gossett jeffrey c sent thursday december escapenumber escapenumber escapenumber escapenumber pm to palmer b scott subject fw part escapenumber deals to move original message from richardson stacey sent thursday december escapenumber escapenumber escapenumber escapenumber pm to gossett jeffrey c cc hodge jeffrey t theriot kim s anderson bridgette hilliard marlene subject part escapenumber deals to move jeff here are some more deals to move into the bankruptcy books i will be out of the office tomorrow but bridgette anderson and marlene hilliard will be working on them if you have any questions thanks sbr xescapenumber', 0)


In [19]:
def get_tokenize(word):
    word = []
    for index, row in word.iterrows():
        word = tokenize_word(row["text"])
        if (len(word) > 0 and len(word.shape) > 0): word.append((example, row["label"]))
    return examples      

In [20]:
def train_trigram(trigram_model, trigram_dataset):

    criteria = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(trigram_model.parameters())

    softmax = nn.Softmax(dim=2)

    epochs = 3
    batch_size = 32
    print_frequency = 1000


    train_dataloader = torch.utils.data.DataLoader(trigram_dataset, batch_size=batch_size, shuffle=True)

    for i in range(epochs):
        print('### Epoch: ' + str(i+1) + ' ###')

        trigram_model.train()
        avg_loss = 0

        for step, data in enumerate(train_dataloader):

            x, y = data

            x = x.unsqueeze(1)

            optimizer.zero_grad()

            model_output = trigram_model(x)
            model_output_probabilities = softmax(model_output)

            loss = criteria(model_output_probabilities.squeeze(1), y)

            loss.backward()
            optimizer.step()

            avg_loss += loss.item()
            if step % print_frequency == 1:
                print('epoch: {} batch: {} loss: {}'.format(
                    i,
                    step,
                    avg_loss / print_frequency
                ))
                avg_loss = 0
    plt.plot(losses)
    plt.xlabel('Step')
    plt.ylabel('Loss')
    plt.title('Loss vs Step')
    plt.show()

    return [model, best_model_sd]

In [21]:
#adapted from Homework 3
def train_lstm_classification(model, train_dataset, valid_dataset, epochs=10, batch_size=32, learning_rate=.001, print_frequency=25):

    criteria = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


    epochs = epochs
    batch_size = batch_size
    print_frequency = print_frequency

    
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)



    best_accuracy = 0.0
    best_model_sd = None

    for i in range(epochs):
        print('### Epoch: ' + str(i+1) + ' ###')
    
        model.train()
        
        print(type (train_dataloader))

        avg_loss = 0

        for step, data in enumerate(train_dataloader):

            x, y = data

            optimizer.zero_grad()

            model_output = model(x, x_lengths)

            loss = criteria(model_output.squeeze(1), y.float())

            loss.backward()
            optimizer.step()

            avg_loss += loss.item()

            if step % print_frequency == (print_frequency - 1):
                print('epoch: {} batch: {} loss: {}'.format(
                    i,
                    step,
                    avg_loss / print_frequency
                ))
                avg_loss = 0

        print('Evaluating...')
        model.eval()
        with torch.no_grad():
            acc = predict(model, valid_dataloader)
            if acc > best_accuracy:
                best_model_sd = copy.deepcopy(model.state_dict())
                best_accuracy = acc

    return model.state_dict(), best_model_sd

In [22]:
class LSTM(nn.Module):
 
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers,batch_first=True)
        self.output_layer = nn.Linear(hidden_size, output_size)

        #adapted from Homework 3
    def forward(self, input_batch, input_lengths):
        print('Input batch shape: {}'.format(input_batch.shape))
        embedded_input = self.embedding(input_batch)
        
        print('Embedded input shape: {}'.format(embedded_input.shape))
        packed_input = pack_padded_sequence(embedded_input, input_lengths, batch_first=True, enforce_sorted=False)
        
        packed_output, (hn, cn) = self.lstm(packed_input)
        
        hn_view = hn.view(self.lstm.num_layers, self.num_directions, input_batch.shape[0], self.lstm.hidden_size)               # Reshape hn for clarity -- first dimension now represents each layer (total set by num_lstm_layers)
        print('hn_view input shape: {}'.format(hn_view.shape))
        
        hn_view_last_layer = hn_view[-1]                                                                                        # Taking the last layer for our final LSTM output
        print('hn_view_last_layer input shape: {}'.format(hn_view_last_layer.shape))
        
        hn_cat = torch.cat([hn_view_last_layer[-2, :, :], hn_view_last_layer[-1, :, :]], dim=1)                                 # Each layer has two directions. We want to use both of these vectors, so concatenate them
        print('hn_cat input shape: {}'.format(hn_cat.shape))
        
        hid = self.relu(self.hidden_1(hn_cat))
        print('hid input shape: {}'.format(hid.shape))
        
        output = self.hidden_2(hid)
        print('output input shape: {}'.format(output.shape))
    
        return output

In [23]:
spam_train = dataset('./datasets/SpamHam/train.csv')
model = LSTM(200, 100, 16, 8)
model, best_model = train_lstm_classification(model, 300, 200, batch_size=128, epochs=1)

### Epoch: 1 ###
<class 'torch.utils.data.dataloader.DataLoader'>


TypeError: object of type 'int' has no len()